In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the neural network architecture
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

batch_size = 64

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Model, loss function, and optimizer
input_size = 28 * 28  # MNIST image size
hidden_size = 128
output_size = 10  # 10 classes for digits 0-9

model = SimpleNN(input_size, hidden_size, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Training
epochs = 10

for epoch in range(epochs):
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Batch [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
            running_loss = 0.0
    scheduler.step()

# Testing
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")


Epoch [1/10], Batch [100/938], Loss: 0.9410
Epoch [1/10], Batch [200/938], Loss: 0.4461
Epoch [1/10], Batch [300/938], Loss: 0.3673
Epoch [1/10], Batch [400/938], Loss: 0.3220
Epoch [1/10], Batch [500/938], Loss: 0.3028
Epoch [1/10], Batch [600/938], Loss: 0.2749
Epoch [1/10], Batch [700/938], Loss: 0.2665
Epoch [1/10], Batch [800/938], Loss: 0.2405
Epoch [1/10], Batch [900/938], Loss: 0.2484
Epoch [2/10], Batch [100/938], Loss: 0.1991
Epoch [2/10], Batch [200/938], Loss: 0.1894
Epoch [2/10], Batch [300/938], Loss: 0.1870
Epoch [2/10], Batch [400/938], Loss: 0.1947
Epoch [2/10], Batch [500/938], Loss: 0.1659
Epoch [2/10], Batch [600/938], Loss: 0.1545
Epoch [2/10], Batch [700/938], Loss: 0.1543
Epoch [2/10], Batch [800/938], Loss: 0.1554
Epoch [2/10], Batch [900/938], Loss: 0.1568
Epoch [3/10], Batch [100/938], Loss: 0.1343
Epoch [3/10], Batch [200/938], Loss: 0.1321
Epoch [3/10], Batch [300/938], Loss: 0.1200
Epoch [3/10], Batch [400/938], Loss: 0.1292
Epoch [3/10], Batch [500/938], L